### Working with Dictionaries (advanced, but not too much)

This notebook adds a few powerful patterns on top of the basics: fast membership tests, safer lookups with defaults, counting/accumulating, merging, safe deletion, views/set-ops, and common creation/transformation tricks.

## Membership tests are on keys

In [1]:
data = {
    'open': 100,
    'high': 110,
    'low': 95,
    'close': 110
}

print('open' in data)     # True
print('volume' in data)   # False
print('volume' not in data)  # True

True
False
True


## Clearing, size, shallow vs deep copy

In [2]:
original = {'a': [1, 2], 'b': [3]}
shallow = original.copy()          # same as dict(original)

from copy import deepcopy
deep = deepcopy(original)

original['a'].append(99)
print('original :', original)
print('shallow  :', shallow)   # shares inner lists -> mutated
print('deep     :', deep)      # fully independent

print('len(original) =', len(original))
tmp = {'x': 1, 'y': 2}
tmp.clear()
print('cleared  :', tmp)

original : {'a': [1, 2, 99], 'b': [3]}
shallow  : {'a': [1, 2, 99], 'b': [3]}
deep     : {'a': [1, 2], 'b': [3]}
len(original) = 2
cleared  : {}


## Creating dictionaries quickly

In [3]:
d1 = dict(high=100, low=95)   # keyword-style keys
print(d1)

# fromkeys: all keys share the same value object
d2 = dict.fromkeys(['open', 'high', 'low', 'close'], 0)
print(d2)

# ⚠️ Pitfall: mutable default is shared
d3 = dict.fromkeys('abc', [])
d3['a'].append(1)
print('mutable fromkeys pitfall:', d3)   # lists shared!

# Proper way when needing independent lists
d4 = {k: [] for k in 'abc'}
d4['a'].append(1)
print('fixed                        ', d4)

{'high': 100, 'low': 95}
{'open': 0, 'high': 0, 'low': 0, 'close': 0}
mutable fromkeys pitfall: {'a': [1], 'b': [1], 'c': [1]}
fixed                         {'a': [1], 'b': [], 'c': []}


## Idiomatic lookups with defaults: get(), setdefault(), and a light taste of defaultdict

- `dict.get(key, default)` — read with a fallback; **does not** insert.
- `dict.setdefault(key, default)` — read-or-insert default **and** return it.
- `collections.defaultdict` — like `setdefault` at the dict level using a factory (shown briefly).

In [4]:
transactions = [
    {'item': 'widget',  'trans_type': 'sale',   'quantity': 10},
    {'item': 'widget',  'trans_type': 'sale',   'quantity': 5},
    {'item': 'widget',  'trans_type': 'refund', 'quantity': 2},
    {'item': 'license', 'trans_type': 'sale',   'quantity': 1},
    {'item': 'license', 'trans_type': 'sale',   'quantity': 1},
    {'item': 'license', 'trans_type': 'refund', 'quantity': 1},
]

# Using get() for totals
total_sold = {}
for t in transactions:
    if t['trans_type'] == 'sale':
        item = t['item']
        total_sold[item] = total_sold.get(item, 0) + t['quantity']
print('total sold      :', total_sold)

# Using setdefault() for grouping
by_item = {}
for t in transactions:
    by_item.setdefault(t['item'], []).append(t)
print('grouped keys    :', {k: len(v) for k, v in by_item.items()})

# A brief look at defaultdict (optional, similar effect)
from collections import defaultdict
group_dd = defaultdict(list)
for t in transactions:
    group_dd[t['item']].append(t)
print('defaultdict keys:', {k: len(v) for k, v in group_dd.items()})

total sold      : {'widget': 15, 'license': 2}
grouped keys    : {'widget': 3, 'license': 3}
defaultdict keys: {'widget': 3, 'license': 3}


## Merging dictionaries: update(), union operator `|`, and unpacking

- `d.update(other)` mutates `d` in-place.
- `d3 = d1 | d2` (Python ≥ 3.9) returns a **new** dict. Right side wins on key collisions.
- `{**d1, **d2}` also builds a new dict (earlier Python compatibility).

In [5]:
d1 = {'a': 1, 'b': 2}
d2 = {'b': 200, 'c': 3}

m1 = d1.copy(); m1.update(d2)   # mutate copy
print('update   ->', m1)

m2 = d1 | d2                     # new dict, Python 3.9+
print('| union  ->', m2)

m3 = {**d1, **d2}                # unpacking
print('unpacking->', m3)

# In-place union
x = {'x': 1}
x |= {'x': 9, 'y': 2}
print('|= result ->', x)

update   -> {'a': 1, 'b': 200, 'c': 3}
| union  -> {'a': 1, 'b': 200, 'c': 3}
unpacking-> {'a': 1, 'b': 200, 'c': 3}
|= result -> {'x': 9, 'y': 2}


## Safe deletion: del, pop with default, popitem (LIFO)

In [6]:
d = {'a': 1, 'b': 2, 'c': 3}

# pop returns the value and can take a default (avoids KeyError)
val = d.pop('b', None)
print('popped b ->', val, 'remaining:', d)

# popitem removes and returns the last inserted (Python 3.7+)
k, v = d.popitem()
print('popitem ->', (k, v), 'remaining:', d)

# del raises if missing
try:
    del d['missing']
except KeyError as e:
    print('del missing -> KeyError:', e)

popped b -> 2 remaining: {'a': 1, 'c': 3}
popitem -> ('c', 3) remaining: {'a': 1}
del missing -> KeyError: 'missing'


## Dictionary views: dynamic and set-like (intersection, diff, etc.)

In [7]:
a = {'x': 1, 'y': 2, 'z': 3}
b = {'w': 0, 'y': 20, 'z': 30}
print('keys(a) & keys(b) ->', a.keys() & b.keys())
print('keys(a) - keys(b) ->', a.keys() - b.keys())
print('values view is dynamic:')
vals = a.values()
print(list(vals))
a['new'] = 99
print(list(vals))  # reflects change

keys(a) & keys(b) -> {'y', 'z'}
keys(a) - keys(b) -> {'x'}
values view is dynamic:
[1, 2, 3]
[1, 2, 3, 99]


## Transforming and filtering: comprehensions

In [8]:
prices = {'banana': 2.5, 'apple': 1.2, 'pear': 2.1, 'cherry': 5.0}

# Apply a 10% discount to items priced >= 2.0
discounted = {k: (v * 0.9 if v >= 2.0 else v) for k, v in prices.items()}
print('discounted:', discounted)

# Keep only keys starting with 'p'
p_only = {k: v for k, v in prices.items() if k.startswith('p')}
print('p_only    :', p_only)

discounted: {'banana': 2.25, 'apple': 1.2, 'pear': 1.8900000000000001, 'cherry': 4.5}
p_only    : {'pear': 2.1}


## Accumulation patterns: counting and netting with a single pass

In [9]:
events = ['hit', 'miss', 'hit', 'hit', 'miss']
counts = {}
for e in events:
    counts[e] = counts.get(e, 0) + 1
print('counts:', counts)

# Net quantities (sales as +, refunds as -) in one pass
net = {}
for t in transactions:
    q = t['quantity'] * (1 if t['trans_type'] == 'sale' else -1)
    net[t['item']] = net.get(t['item'], 0) + q
print('net   :', net)

counts: {'hit': 3, 'miss': 2}
net   : {'widget': 13, 'license': 1}


## Robust nested get (avoid KeyError chains)

In [10]:
order = {
    'id': 123,
    'customer': {
        'name': 'Ada',
        'address': {'city': 'London'}
    }
}

city = order.get('customer', {}).get('address', {}).get('city', 'Unknown')
print('city:', city)

missing_zip = order.get('customer', {}).get('address', {}).get('zip', None)
print('zip :', missing_zip)

city: London
zip : None
